In [19]:
import os
from typing import Any, List, Union

from langchain_community.chat_models import ChatLiteLLM
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.output_parsers.base import T
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.agents import Tool, LLMSingleActionAgent, initialize_agent, AgentExecutor

from langchain.agents.agent import AgentOutputParser
from langchain.agents.conversational.prompt import FORMAT_INSTRUCTIONS
from langchain_core.runnables import RunnableSequence


In [2]:
def search(query):
    # This function should implement the logic to search for products
    # For now, it just returns a placeholder response
    return f"Searching for: {query}"

tools = [
    Tool(
            name="ProductSearch",
            func=search,
            description="useful for when you need to answer questions about product information or services offered, availability and their costs.",
        )
]
tool_names = [tool.name for tool in tools]

In [3]:
import re

class SalesConvoOutputParser(AgentOutputParser):
    ai_prefix: str = "AI"  # change for salesperson_name
    verbose: bool = False

    def get_format_instructions(self) -> str:
        return """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```"""

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        if self.verbose:
            print("TEXT")
            print(text)
            print("-------")
        regex = r"Action: (.*?)[\n]*Action Input: (.*)"
        match = re.search(regex, text)
        if not match:
            return AgentFinish(
                {"output": text.split(f"{self.ai_prefix}:")[-1].strip()}, text
            )
        action = match.group(1)
        action_input = match.group(2)
        return AgentAction(action.strip(), action_input.strip(" ").strip('"'), text)

output_parser = SalesConvoOutputParser()

In [14]:
text = """AI:lalaei f"""
res = output_parser.parse(text)
print(res)

return_values={'output': 'lalaei f'} log='AI:lalaei f'


In [4]:
class CustomPromptTemplateForTools(PromptTemplate):

    template: str
    tools: list[Tool]

    def format(self, **kwargs: Any) -> str:
        kwargs = self._merge_partial_and_user_variables(**kwargs)
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        kwargs["format_instruction"] = output_parser.get_format_instructions()
        return self.template.format(**kwargs)

prompt = CustomPromptTemplateForTools(
    template="You are an AI Sales agent, sell me anything，Here are the questions you must answer:{input}\n\nAnd you has access to the following tools: \n{tools} \n {format_instruction} \n{intermediate_steps}",
    tools=tools,
    input_variables=["input", "intermediate_steps","tools"],
    partial_variables={"format_instruction": output_parser.get_format_instructions()},
    validate_template=True
)
# print(prompt)
print(prompt.format(input="Who are you?", intermediate_steps=[]))

You are an AI Sales agent, sell me anything，Here are the questions you must answer:Who are you?

And you has access to the following tools: 
ProductSearch: useful for when you need to answer questions about product information or services offered, availability and their costs. 
 To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
``` 
[]


In [5]:
os.environ["DASHSCOPE_API_KEY"] = 'sk-36c7f4e71d904a35afab94dc68747d48'
llm = ChatLiteLLM(model="openai/qwen-max",
                  api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
                  api_key=os.environ.get('DASHSCOPE_API_KEY'))

C:\Users\ww\AppData\Local\Temp\ipykernel_9380\4053257069.py:2: LangChainDeprecationWarning: The class `ChatLiteLLM` was deprecated in LangChain 0.3.24 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-litellm package and should be used instead. To use it run `pip install -U :class:`~langchain-litellm` and import as `from :class:`~langchain_litellm import ChatLiteLLM``.
  llm = ChatLiteLLM(model="openai/qwen-max",


In [25]:
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
# llm_chain = prompt | llm

In [23]:
output = llm_chain.invoke({"input":"Who are you?","intermediate_steps":[],"tools":""}, return_intermediate_steps=True)
print(output)

content="Thought: Do I need to use a tool? No\nQwen: Hello! I'm Qwen, your dedicated AI Sales Assistant, here to assist you with finding the perfect product or service that meets your needs. Whether it's technology, home essentials, or something unique, I can help guide you through our wide selection and provide all the information you require. How can I assist you today? Is there anything specific you're looking for?" additional_kwargs={} response_metadata={'token_usage': Usage(completion_tokens=89, prompt_tokens=179, total_tokens=268, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None)), 'model': 'openai/qwen-max', 'finish_reason': 'stop', 'model_name': 'openai/qwen-max'} id='run--7acb1e85-e37c-4d7e-b2c8-d70e7d636742-0' usage_metadata={'input_tokens': 179, 'output_tokens': 89, 'total_tokens': 268}


In [26]:
sales_agent_with_tools = LLMSingleActionAgent(
                llm_chain=llm_chain,
                output_parser=output_parser,
                stop=["\nObservation:"],
                allowed_tools=tool_names,
            )

In [27]:
agent = AgentExecutor.from_agent_and_tools(
    agent=sales_agent_with_tools,
    tools=tools,
    verbose=True,
)

In [28]:
inputs = {
    "input":"Can you tell me about the product B",
    "tools":"",
}
# res = agent.run(inputs)
res = agent.invoke(inputs)
print(res)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an AI Sales agent, sell me anything，Here are the questions you must answer:Can you tell me about the product B

And you has access to the following tools: 
ProductSearch: useful for when you need to answer questions about product information or services offered, availability and their costs. 
 To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
``` 
[]

> Finished chain.
Thought: Do I need to use a tool? Yes
Action: ProductSearch
Action Input: Information about product B, including its features, benefits, and pricing.

Obs

KeyboardInterrupt: 

In [92]:
# Define template
template = "What are the capitals of {country1} and {country2}, respectively?"

# Create a prompt template with `PromptTemplate` object
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "United States of America"  # Pass `partial_variables` in dictionary form
    },
    validate_template=True
)
# print(prompt)
formatted_prompt = prompt.format(country1="France")
print(formatted_prompt)

What are the capitals of France and United States of America, respectively?
